# seq2seq - nmt

In [11]:
from collections import Counter
from datetime import datetime
import pickle
import re

import jieba
from nltk.stem.porter import PorterStemmer
from nltk.translate.bleu_score import corpus_bleu
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf  # 1.0.1
from tensorflow.python.ops import variable_scope

from data_helpers import (buckets, _PAD, _GO, _EOS, _UNK, _PAD_ID, _GO_ID, _EOS_ID, _UNK_ID)

%matplotlib inline
# %load_ext watermark
# %watermark -p tensorflow,numpy -v -m

## 模型参数

In [2]:
local_test = False

if local_test:
    cell_size = 10 
    num_layers = 2  # rnn cell 层数
    embedding_size = 10 
    num_sampled = 300 
else:
    cell_size = 100
    num_layers = 2  # rnn cell 层数
    embedding_size = 100
    num_sampled = 500 

## 加载处理好的语料和词表

In [4]:
with open('^data/data_13w.pkl', 'rb') as f:
    encoder_data, decoder_data, encoder_data_test, decoder_data_test = pickle.load(f)

with open('^data/vocab_13w.pkl', 'rb') as f:
    vocab_enc, vocab_dec = pickle.load(f)

data_sizes = [len(encoder_data[i]) for i in range(len(buckets))]
data_sizes_dec = [len(decoder_data[i]) for i in range(len(buckets))]
assert data_sizes == data_sizes_dec

data_sizes_test = [len(encoder_data_test[i]) for i in range(len(buckets))]
data_sizes_dec_test = [len(decoder_data_test[i]) for i in range(len(buckets))]
assert data_sizes_test == data_sizes_dec_test

print(data_sizes)
print(data_sizes_test)

[59648, 71510]
[2903, 3160]


In [5]:
num_encoder_symbols = len(vocab_enc)
num_decoder_symbols = len(vocab_dec)
num_encoder_symbols, num_decoder_symbols

(7655, 10340)

## 定义 RNN cell


In [6]:
tf.reset_default_graph()

def single_cell():
    return tf.contrib.rnn.GRUCell(cell_size)
cell = single_cell()
if num_layers > 1:
    cell = tf.contrib.rnn.MultiRNNCell([single_cell() for _ in range(num_layers)])

## 创建 placeholders

注意在 bucketing 时, decoder inputs 已经做了首尾的 token: GO 和 EOS.

In [7]:
max_encoder_length, max_decoder_length = buckets[-1]

encoder_placeholders = [
    tf.placeholder(tf.int32, shape=[None], name="encoder_%d" % i)
    for i in range(max_encoder_length)]
decoder_placeholders = [
    tf.placeholder(tf.int32, shape=[None], name="decoder_%d" % i)
    for i in range(max_decoder_length)]
target_placeholders = [
    tf.placeholder(tf.int32, shape=[None], name="target_%d" % i)
    for i in range(max_decoder_length)]
target_weights_placeholders = [
    tf.placeholder(tf.float32, shape=[None], name="decoder_weight_%d" % i)
    for i in range(max_decoder_length)]

##  `model_with_buckets` 构建模型

注: tf 1.0.1 ok. tf 1.1.0 接口有变, 会报错.

In [8]:
def seq2seq_f(encoder_placeholders, decoder_placeholders, do_decode):
    # 可以在这里设置不同的 seq2seq 接口, 比如 embedding_attention_seq2seq
    return tf.contrib.legacy_seq2seq.embedding_attention_seq2seq(
        encoder_placeholders,
        decoder_placeholders,
        cell,
        num_encoder_symbols=num_encoder_symbols,
        num_decoder_symbols=num_decoder_symbols,
        embedding_size=embedding_size,
        output_projection=output_projection,
        feed_previous=do_decode)

def sampled_loss(labels, logits):
    labels = tf.reshape(labels, [-1, 1])
    return tf.nn.sampled_softmax_loss(
        weights=softmax_w_t,
        biases=softmax_b,
        labels=labels,
        inputs=logits,
        num_sampled=num_sampled,
        num_classes=num_decoder_symbols)

softmax_w_t = tf.get_variable("proj_w", [num_decoder_symbols, cell_size], dtype=tf.float32)
softmax_w = tf.transpose(softmax_w_t)
softmax_b = tf.get_variable("proj_b", [num_decoder_symbols], dtype=tf.float32)
output_projection = (softmax_w, softmax_b)

outputs, losses = tf.contrib.legacy_seq2seq.model_with_buckets(
    encoder_placeholders, decoder_placeholders, target_placeholders,
    target_weights_placeholders, buckets, lambda x, y: seq2seq_f(x, y, False),
    softmax_loss_function=sampled_loss)

## 准备 feed 数据

注意 embedding_rnn_seq2seq 接收的 encoder_inputs 形状为 `A list of 1D int32 Tensors of shape [batch_size]`. 为此, 可以用 `list(zip(*lst))` 来对 nested list 进行"转置", 得到需要的形状.

In [9]:
def left_shift(decoder_inputs):
    """generate targets grom decoder_inputs"""
    return [list(input_[1:]) + [_PAD_ID] for input_ in decoder_inputs]

def get_bucket_inputs(encoder_data, decoder_data, bucket_id):
    encoder_inputs = encoder_data[bucket_id]
    decoder_inputs = decoder_data[bucket_id]
    return (encoder_inputs, decoder_inputs)

def get_batch_inputs(encoder_data, decoder_data, bucket_id, batch_start, batch_size):
    encoder_inputs = encoder_data[bucket_id][batch_start : batch_start+batch_size]
    decoder_inputs = decoder_data[bucket_id][batch_start : batch_start+batch_size]
    return (encoder_inputs, decoder_inputs)

def generate_feed_dict(inputs_tuple, encoder_size, decoder_size):
    """对 inputs 做转置, 并喂给 placeholder 列表, 得到 feed_dict"""
    encoder_inputs, decoder_inputs = inputs_tuple
    encoder_inputs = list(zip(*encoder_inputs))
    target_inputs = list(zip(*left_shift(decoder_inputs)))
    decoder_inputs = list(zip(*decoder_inputs)) 
    
    feed_dict = dict()
    # Prepare input data
    for i in range(encoder_size):
        # 这里用 placeholder 或者 placeholder.name 都可以
        feed_dict[encoder_placeholders[i].name] = np.asarray(encoder_inputs[i], dtype=int)
    for i in range(decoder_size):
        feed_dict[decoder_placeholders[i].name] = np.asarray(decoder_inputs[i], dtype=int)
        feed_dict[target_placeholders[i].name] = np.asarray(target_inputs[i], dtype=int)        
        # 这里使用 weights 把 <PAD> 的损失屏蔽了
        feed_dict[target_weights_placeholders[i].name] = np.asarray(
            [float(idx != _PAD_ID) for idx in target_inputs[i]], dtype=float)
    return feed_dict

## 训练


In [9]:
# 训练相关参数
epochs = 300
print_loss_every = 5
learning_rate = 0.1
batch_size = 50

# 把不同 bucket 的 loss 分别传给 optimizer, 得到不同的 train_step.
train_steps = [tf.train.AdagradOptimizer(learning_rate).minimize(losses[i]) 
               for i in range(len(buckets))]

sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [10]:
# Training
try:
    print('{:14}'.format('Time') +
          ' Epoch ' +
          ' '.join(['bucket-{}'.format(i) for i in range(len(buckets))]))
    for i in range(epochs):
        if i % print_loss_every == 0:
            print('\n{} {: 4d}'.format(str(datetime.now())[5:-7], 
                                       i), end=' ')

        for bucket_id in range(len(buckets)):
            cur_data_size = data_sizes[bucket_id]
            if cur_data_size == 0:
                continue  # 某个 bucket 为空的特殊情形
            encoder_size, decoder_size = buckets[bucket_id]
            
            # 输出 loss 过程信息
            if i % print_loss_every == 0:
                bucket_inputs = get_bucket_inputs(encoder_data, decoder_data, bucket_id)
                bucket_feed = generate_feed_dict(bucket_inputs, encoder_size, decoder_size)
                loss_val = sess.run(losses[bucket_id], feed_dict=bucket_feed)
                print('{: 8.4f}'.format(loss_val), end=' ')
            
            # 训练
            for batch_start in range(0, cur_data_size, batch_size):
                batch_inputs = get_batch_inputs(
                    encoder_data, decoder_data, bucket_id, batch_start, batch_size)
                batch_feed = generate_feed_dict(batch_inputs, encoder_size, decoder_size)
                sess.run(train_steps[bucket_id], feed_dict=batch_feed)
except KeyboardInterrupt:
    print('\nKeyboardInterrupt')

# plt.plot(range(0, i, print_loss_every), loss_history);

Time           Epoch bucket-0 bucket-1

05-16 16:39:37    0   8.2900   5.1339 
05-16 17:03:02    5   1.9113   2.6406 
05-16 17:26:21   10   1.3782   2.0690 
05-16 17:49:26   15   1.1184   1.8494 
05-16 18:12:31   20   1.0265   1.7606 
05-16 18:35:38   25   0.9266   1.6591 
05-16 18:58:52   30   0.9358   1.6388 
05-16 19:22:06   35   0.8697   1.6019 
05-16 19:45:12   40   0.8602   1.5540 
05-16 20:08:29   45   0.8384   1.5853 
05-16 20:31:33   50   0.8289   1.5236 
05-16 20:54:39   55   0.8224   1.5609 
05-16 21:17:43   60   0.8483   1.4962 
05-16 21:40:44   65   0.8166   1.5114 
05-16 22:03:44   70   0.8236   1.5825 
05-16 22:26:44   75   0.8087   1.5266 
05-16 22:49:52   80   0.8049   1.5184 
05-16 23:13:03   85   0.7548   1.5002 
05-16 23:36:03   90   0.7928   1.5417 
05-16 23:59:11   95   0.7958   1.5213 
KeyboardInterrupt


用 `embedding_attention_seq2seq` 替换 `embedding_rnn_seq2seq` 后, 单步计算时间变长, 但 cost 也下降得更快.

In [13]:
# 保存模型参数
saver = tf.train.Saver()
saver.save(sess, "./^tmp/model_v04.ckpt")

'./tmp/model_v04.ckpt'

## 测试

decoding 过程中, 因为 feed_previous 为 true, 所以 `decoder_inputs` 只用到第一个元素, 后面的都不需要.

In [10]:
# 读取模型参数
saver = tf.train.Saver()
sess = tf.Session()
saver.restore(sess, "./^tmp/model_v04.ckpt")

In [12]:
num_examples_to_print = 50

def cut_at_eos(sentence):
    if _EOS in sentence:        
        return sentence[:sentence.index(_EOS)]
    else:
        return sentence

def no_prepending_pad(sentence):
    for i in range(len(sentence)):
        if sentence[i] != _PAD:
            return sentence[i:]

list_of_references = []
hypotheses = []

with variable_scope.variable_scope(variable_scope.get_variable_scope(), reuse=True):
    outputs, losses = tf.contrib.legacy_seq2seq.model_with_buckets(
        encoder_placeholders, decoder_placeholders, target_placeholders,
        target_weights_placeholders, buckets, lambda x, y: seq2seq_f(x, y, True),
        softmax_loss_function=sampled_loss)
    
    if output_projection is not None:
        for bucket_id in range(len(buckets)):
            outputs[bucket_id] = [
                tf.matmul(output, output_projection[0]) + output_projection[1]
                for output in outputs[bucket_id]]

    for bucket_id in range(len(buckets)):
        print('\n** bucket {}'.format(bucket_id))
        cur_data_size = data_sizes_test[bucket_id]
        print('test examples: {}'.format(cur_data_size))
        if cur_data_size == 0:
            continue  # 某个 bucket 为空的特殊情形

        encoder_size, decoder_size = buckets[bucket_id]
        bucket_inputs_test = get_bucket_inputs(encoder_data_test, decoder_data_test, bucket_id)
        bucket_feed = generate_feed_dict(bucket_inputs_test, encoder_size, decoder_size)

        loss_val = sess.run(losses[bucket_id], feed_dict=bucket_feed)
        print('loss = {: 8.4f}'.format(loss_val))
        
        output_bucket = np.zeros((cur_data_size, decoder_size), dtype=int)  
        # output_bucket 用于记录当前bucket输出值, 形状是 outputs 的"转置"
    
        for i in range(decoder_size):
            prob = outputs[bucket_id][i]  # 第i个词的概率输出
            output_bucket[:, i] = np.argmax(sess.run(prob, feed_dict=bucket_feed), axis=1)
        
        for j in range(cur_data_size):
            sen = [vocab_dec[output_bucket[j, k]] for k in range(decoder_size)]
            input_ = [vocab_enc[i] for i in encoder_data_test[bucket_id][j]]
            target_ = [vocab_dec[i] for i in decoder_data_test[bucket_id][j][1:]]
            input_ = no_prepending_pad(input_)
            sen = cut_at_eos(sen)
            target_ = cut_at_eos(target_)
            
            list_of_references.append([target_])
            hypotheses.append(sen)

            if j <= num_examples_to_print:  # 只打印前几个例子
                print(' input: ', ' '.join(input_))
                print('output: ', ' '.join(sen))
                print('target: ', ' '.join(target_))
                print()


** bucket 0
test examples: 2903
loss =   5.3535
 input:  II . organ OF the meet
output:  二 . 会议 安排 . 1 - 11 2
target:  二 . 本次 会议 的 组织 事项

 input:  through improv ocean govern ( agenda item 7 )
output:  通过 改进 能源 的 议程 ( 议程 项目 7 )
target:  的 实施 工作 ( 议程 项目 7 )

 input:  it work should be reinforc .
output:  在 此 方面 开始 加强 工作 应是 加强 的 。
target:  应 设法 加强 该 协调 处 的 工作 。

 input:  xii . adopt OF the report OF the meet
output:  十二 . 通过 会议 报告
target:  十二 . 通过 本次 会议 报告

 input:  mainstream of the global programm of action
output:  将 采取 全球 行动 方案 纳入 全球 方案
target:  将 《 全球 行动 纲领 》 作为 主流 事项

 input:  financ of the global programm of action
output:  全球 行动 方案 的 经费 筹措
target:  《 全球 行动 纲领 》 的 筹资

 input:  for the protect OF the marin environ from
output:  森林 及 森林 带来 森林 缔约方 的 保护 委员会
target:  第一次 政府 间 审查会议 联席 主席 的 结论

 input:  C . the strateg action plan on municip wastewat
output:  C . 《 荒漠化 团 行动计划 》 的 行动计划
target:  C . 城市 废水 问题 战略 行动计划

 input:  In thi regard , it aim to :
output:  在 这方面 , 其 目的 : 旨在 :
target

训练时的 loss, 是在 feed_previous 为假时算得的.  
用同样的模型参数, 在 decode 时, 因为 feed_previous 为真, 算得的 loss 会大很多.  

In [14]:
bleu = corpus_bleu(list_of_references, hypotheses)
print('BLEU score: {:.4f}'.format(bleu))

BLEU score: 0.2820


## ChangeLog

* v0.0
  * 读入 6w 行, 筛选后 data sizes = 5315, 9982, 13714, 12743
  * buckets = [(7, 7), (15, 15), (25, 25), (35, 35)]
  * 词表大小: 24468 22505 (min freq = 2, 2)
  * train cost: 0.4598   0.8442   2.2040   2.7353  (步长=1, 训练时间 4h)
  * test cost: 6.1954, 9.2254, 10.3698, 10.5111
* v0.1
  * 读入 80w 行, 筛选后 data sizes = 28860, 44106
  * buckets = [(11, 11), (15, 15)]
  * 词表大小: 5402 6815 (min freq = 7, 8)
  * train cost: 1.1266   2.2595 (步长=1, 训练时间 7h) -> 步长取大了
  * test cost: 5.7415, 6.9102
* v0.2
  * 改用 `embedding_attention_seq2seq`
  * 步长: 0.3
  * train cost: 1.0095   2.7291 (训练时间 6.5h)
  * test cost: 5.0621, 8.4917
* v0.3
  * 步长: 0.1
  * train cost: 0.9406   1.8938 (time = 2.5 h)
  * test cost: 4.5514, 7.4168
    * feed_previous 改为 True (仅用于查看检验), test cost: 2.1128, 3.4633
  * BLEU: 0.2112
* v0.4
  * 有效数据加倍 -> [59648, 71510]
  * train cost: 0.7958   1.5213 (time = 7h)
  * test cost: 5.3535, 6.5161
  * BLEU: 0.2820